In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import re, string, unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
# import smart_open
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import math
from math import log
from numpy.random import default_rng, rand


In [268]:
train_data = pd.read_csv("../part2/train_data.tsv", sep='\t')
validation_data = pd.read_csv("../part2/validation_data.tsv", sep='\t')


In [156]:
train_data.head(10)

,qid,pid,queries,passage,relevancy
0,188714,1000052,foods and supplements to lower blood sugar,Watch portion sizes: ■ Even healthy foods will...,0.0
1,995526,1000094,where is the federal penitentiary in ind,It takes THOUSANDS of Macy's associates to bri...,0.0
2,660957,1000115,what foods are good if you have gout?,The good news is that you will discover what g...,0.0
3,837202,1000252,what is the nutritional value of oatmeal,"Oats make an easy, balanced breakfast. One cup...",0.0
4,130825,1000268,definition for daring,Such a requirement would have three desirable ...,0.0
5,408149,1000288,is dhgate a scam,If you think you ve been targeted by a counter...,0.0
6,1019649,1000419,what study for mets to brain,Sorry he's having so much pain. The reason tha...,0.0
7,1099065,1000436,how far deep to plant beet early wonder,"The simplest way, and my preference, is to roa...",0.0
8,1084910,1000466,what disease do roof rats cause,1 A cage trap baited with peanut butter or a s...,0.0
9,959083,1000479,when was niagara falls created,"Bulbar Onset – ALS. ALS is like Niagara Falls,...",0.0


In [164]:
unique_qids_tr = np.unique(train_data.qid.values)
unique_qids_tr.shape

(4590,)

In [190]:
qid = unique_qids_tr[2]
train_data[(train_data['qid']==qid)].count()
qid

qid          1000
pid          1000
queries      1000
passage      1000
relevancy    1000
dtype: int64

2235

In [157]:
train_data.shape

(4364339, 5)

In [158]:
validation_data.head(10)

,qid,pid,queries,passage,relevancy
0,1082792,1000084,what does the golgi apparatus do to the protei...,"Start studying Bonding, Carbs, Proteins, Lipid...",0.0
1,995825,1000492,where is the graphic card located in the cpu,"For example, a “PC Expansion Card” maybe the j...",0.0
2,995825,1000494,where is the graphic card located in the cpu,The Common Cards & Buses. The most common type...,0.0
3,1091246,1000522,property premises meaning,The occurrence of since tells us that the firs...,0.0
4,1047854,1000585,what is printing mechanism,Windows desktop applications Develop Desktop t...,0.0
5,991832,1000599,who discovered the element carbon,1. 1 a nonmetallic element existing in the th...,0.0
6,185299,1000647,fastest cell phone processor,Tips for calling a cell phone in Greece: To ca...,0.0
7,574730,1000663,what are the three monetary policy tools of th...,"Federal Reserve updates including rates, news ...",0.0
8,1085008,1000675,what did maria theresa do for the serfs,"In this feudal system, the king awarded land g...",0.0
9,609628,1000771,what county is mitchell south dakota in,South Dakota: According to our research of Sou...,0.0


In [298]:
unique_qids_val = np.unique(validation_data.qid.values)
unique_qids_val.shape

(1148,)

In [159]:
train_data.shape
validation_data.shape

(4364339, 5)

(1103039, 5)

# Under Sampling

## train data

use all qids and take all relative pids and say 200 non-relative

In [277]:
unique_qids_tr = np.unique(train_data.qid.values)
train_data_sample = train_data[train_data['relevancy'] == 1.0]

In [274]:
unique_qids_tr.shape

(4590,)

In [278]:
train_data_sample

,qid,pid,queries,passage,relevancy
3124,709560,1050990,what is all in basic metabolic panel,Basic Metabolic Panel. The basic metabolic pan...,1.0
3514,1098846,1057355,how is rose quartz used etc,"In the psychic and spiritual realms, rose quar...",1.0
5867,182081,1091839,euless texas is in what county?,"Euless, Texas. Euless (/ˈjuːlɪs/ YOO-liss) is ...",1.0
8194,895932,1130834,what sizes do bar stools come in,These are commonly called “spectator stools”. ...,1.0
8861,1021679,1143017,which type of blood vessel carries blood away ...,Arteries. The arteries are major blood vessels...,1.0
...,...,...,...,...,...
4343822,401287,860900,is a written prescription required for hydroco...,Refills of hydrocodone combination products wi...,1.0
4352626,541272,876066,was wilson a good president,"Woodrow Wilson (1856-1924), the 28th U.S. pres...",1.0
4356455,845529,882642,what is the salary range of a dentist,Dentist Salary. (United States). The average s...,1.0
4360007,850361,926854,what is the temperature in washington,July is the hottest month in Washington DC wit...,1.0


In [279]:
train_data[(train_data['qid'] == 709560) & (train_data['relevancy'] == 0.0)].sample(n = 100)

,qid,pid,queries,passage,relevancy
4014418,709560,319261,what is all in basic metabolic panel,Details. CMP-14 - The Comprehensive Metabolic ...,0.0
2316321,709560,7212670,what is all in basic metabolic panel,CMP blood test. The CMP blood test (comprehens...,0.0
1229926,709560,5276552,what is all in basic metabolic panel,The comprehensive metabolic panel (CMP) is a f...,0.0
1855968,709560,7600486,what is all in basic metabolic panel,A CMP includes all of the components of a BMP ...,0.0
465257,709560,8620886,what is all in basic metabolic panel,This field is utilized for any field that requ...,0.0
...,...,...,...,...,...
3155512,709560,5085576,what is all in basic metabolic panel,- is the basic structural and functional unit ...,0.0
3685020,709560,5786980,what is all in basic metabolic panel,The standard size for a patio door depends on ...,0.0
3859637,709560,862099,what is all in basic metabolic panel,"In an emergency, technicians can co the compre...",0.0
4358347,709560,900851,what is all in basic metabolic panel,Note: Serum is the part of blood that doesn't ...,0.0


In [287]:
for qid in unique_qids_tr:
    try:
        temp = train_data[(train_data['qid'] == qid) & (train_data['relevancy'] == 0.0)].sample(n = 100)
    except:
        temp = train_data[(train_data['qid'] == qid) & (train_data['relevancy'] == 0.0)]
    train_data_sample = train_data_sample.append(temp, ignore_index=True)

In [288]:
train_data_sample

,qid,pid,queries,passage,relevancy
0,709560,1050990,what is all in basic metabolic panel,Basic Metabolic Panel. The basic metabolic pan...,1.0
1,1098846,1057355,how is rose quartz used etc,"In the psychic and spiritual realms, rose quar...",1.0
2,182081,1091839,euless texas is in what county?,"Euless, Texas. Euless (/ˈjuːlɪs/ YOO-liss) is ...",1.0
3,895932,1130834,what sizes do bar stools come in,These are commonly called “spectator stools”. ...,1.0
4,1021679,1143017,which type of blood vessel carries blood away ...,Arteries. The arteries are major blood vessels...,1.0
...,...,...,...,...,...
453655,1102400,5804047,why do bears hibernate,"They are based on the real life cave bear, Urs...",0.0
453656,1102400,5350266,why do bears hibernate,Polar bears are the largest land carnivores in...,0.0
453657,1102400,5546084,why do bears hibernate,That connected bears with President Roosevelt....,0.0
453658,1102400,1994885,why do bears hibernate,In this article I’ve gathered all the essentia...,0.0


In [290]:
train_data_sample = train_data_sample.sample(frac=1).reset_index(drop=True)

In [291]:
train_data = train_data_sample.sample(frac=1).reset_index(drop=True)

In [292]:
train_data

,qid,pid,queries,passage,relevancy
0,406386,7869580,is cider vinegar safe,2. It can make your hair shine. Apple cider vi...,0.0
1,1087603,1141888,what are field peas,"Dogs can eat raw or cooked peas, but cooked pe...",0.0
2,149790,3074270,different hindu prayers,"Puja. Hindu worship, or puja, involves images ...",0.0
3,115704,931826,david tepper net worth,Videos Earthquake The Comedian Net Worth 2017 ...,0.0
4,541969,1656275,watt to kilowatt hours,An electric heater rated at 1000 watts (1 kilo...,0.0
...,...,...,...,...,...
453655,1093405,4682759,beechen definition,Related searches for Operational Definitions. ...,0.0
453656,1088718,2728475,weather for avon in,"In English, the name Avon means-the river.The ...",0.0
453657,1085348,7335165,what county is polson montana in?,Average Teacher Salary in Montana : Montana Te...,0.0
453658,155056,3966495,do astronauts sleep on their back,"When you’re suffering from a bad back, it can ...",0.0


## validation data

In [297]:
validation_data_sample = validation_data[validation_data['relevancy'] == 1.0]
validation_data_sample

,qid,pid,queries,passage,relevancy
26,135386,100163,definition of imagination,imagination - the formation of a mental image ...,1.0
4825,244011,1308535,how long do I bake skinless seared chicken bre...,Bake for a short time at a high temperature. B...,1.0
17557,806574,2113439,what is the average monthly car insurance esti...,Monthly auto insurance premiums can range from...,1.0
17914,837372,2137413,what is the official site for free credit reports,How can I get a free copy of my credit report?...,1.0
26978,531142,2730772,typing average words per minute test,The median typist speed is 38 WPM and the mean...,1.0
...,...,...,...,...,...
1088617,137508,7994418,definition of purse,noun. verb. The definition of a purse is a sma...,1.0
1088695,1090395,7999912,sterility meaning,sterility noun [U] us / stəˈrɪl·ɪ·t̬i / › the ...,1.0
1088766,291248,8003933,how many ounces of liquid is in one gallon,There are 128 fluid ounces in 1 US gallon. The...,1.0
1088815,20520,8007226,arbitrage definition,Full Definition of ARBITRAGE. 1. : the nearly ...,1.0


In [299]:
for qid in unique_qids_val:
    try:
        temp = validation_data[(validation_data['qid'] == qid) & (validation_data['relevancy'] == 0.0)].sample(n = 100)
    except:
        temp = validation_data[(validation_data['qid'] == qid) & (validation_data['relevancy'] == 0.0)]
    validation_data_sample = validation_data_sample.append(temp, ignore_index=True)

In [300]:
validation_data_sample

,qid,pid,queries,passage,relevancy
0,135386,100163,definition of imagination,imagination - the formation of a mental image ...,1.0
1,244011,1308535,how long do I bake skinless seared chicken bre...,Bake for a short time at a high temperature. B...,1.0
2,806574,2113439,what is the average monthly car insurance esti...,Monthly auto insurance premiums can range from...,1.0
3,837372,2137413,what is the official site for free credit reports,How can I get a free copy of my credit report?...,1.0
4,531142,2730772,typing average words per minute test,The median typist speed is 38 WPM and the mean...,1.0
...,...,...,...,...,...
114027,1102335,171576,why do people buy cars,Cars as a Symbol in The Great Gatsby-Cars as a...,0.0
114028,1102335,2053529,why do people buy cars,Tariffs are meant so people will buy more dome...,0.0
114029,1102335,8771062,why do people buy cars,eBay and Other Online Auction Sites. Most peop...,0.0
114030,1102335,2292604,why do people buy cars,Return Option Insurance. Before you buy the ca...,0.0


In [301]:
validation_data = validation_data_sample.sample(frac=1).reset_index(drop=True)

In [ ]:
train_data.to_pickle('train_data_sample_original2.pkl')
validation_data.to_pickle('validation_data_sample_original2.pkl')

Select k out of N training queries
Use all passages corresponding to those k queries for the training
This would allow to reduce the size of the training set by (1-(k/N))%, whilst keeping all queries and the corresponding (relevant and irrelevant) passages together.

k = 150

## train data

In [303]:
# k = 400
# unique_qids_tr = np.unique(train_data.qid.values)
# selected_qids = np.random.choice(unique_qids_tr,k)
# selected_qids

In [304]:
# train_data = train_data[train_data['qid'].isin(selected_qids)]


In [305]:
# train_data.reset_index(drop=True, inplace=True)

## Validation Data 

In [306]:
# k = 100
# unique_qids_val = np.unique(validation_data.qid.values)
# selected_qids = np.random.choice(unique_qids_val,k)
# selected_qids

In [307]:
# validation_data = validation_data[validation_data['qid'].isin(selected_qids)]
# validation_data.reset_index(drop=True, inplace=True)
# validation_data

In [223]:
# train_data.to_pickle('train_data_sample_original.pkl')
# validation_data.to_pickle('validation_data_sample_original.pkl')

## Preprocessing

In [3]:
def remove_punctuation(tokens):
    new_tokens = []
    for token in tokens:
        new_token = re.sub(r'[^\w\s]', '', token)
        if new_token != '':
            new_tokens.append(new_token)
    return new_tokens

def remove_stopwords(tokens):
    new_tokens = []
    stopword_set = set(stopwords.words('english'))
    for token in tokens:
        if token not in stopword_set:
            new_tokens.append(token)
    return new_tokens

def lemmatize_verbs(tokens):
    lemmatizer = WordNetLemmatizer()
    root_words = []
    for token in tokens:
        root_word = lemmatizer.lemmatize(token, pos='v')
#         root_word = lemmatizer.lemmatize(token, pos='n')
#         root_word = lemmatizer.lemmatize(token, pos='a')
        root_words.append(root_word)
    return root_words

def remove_numbers(tokens):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    new_tokens = []
    for token in tokens:
        if token.isdigit():
            pass
        else:
            new_tokens.append(token)
    return new_tokens

def preprocessing(passage):
    passage = passage.lower()
    tokens = nltk.word_tokenize(passage)
    tokens = remove_punctuation(tokens)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize_verbs(tokens)
    tokens = remove_numbers(tokens)
    return tokens

In [309]:
train_data['passage_cleaned']=train_data.passage.apply(lambda x: preprocessing(x))
train_data['query_cleaned']=train_data.queries.apply(lambda x: preprocessing(x))

In [310]:
train_data

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned
0,406386,7869580,is cider vinegar safe,2. It can make your hair shine. Apple cider vi...,0.0,"[make, hair, shine, apple, cider, vinegar, use...","[cider, vinegar, safe]"
1,1087603,1141888,what are field peas,"Dogs can eat raw or cooked peas, but cooked pe...",0.0,"[dog, eat, raw, cook, peas, cook, peas, softer...","[field, peas]"
2,149790,3074270,different hindu prayers,"Puja. Hindu worship, or puja, involves images ...",0.0,"[puja, hindu, worship, puja, involve, image, m...","[different, hindu, prayers]"
3,115704,931826,david tepper net worth,Videos Earthquake The Comedian Net Worth 2017 ...,0.0,"[videos, earthquake, comedian, net, worth, dig...","[david, tepper, net, worth]"
4,541969,1656275,watt to kilowatt hours,An electric heater rated at 1000 watts (1 kilo...,0.0,"[electric, heater, rat, watts, kilowatt, opera...","[watt, kilowatt, hours]"
...,...,...,...,...,...,...,...
453655,1093405,4682759,beechen definition,Related searches for Operational Definitions. ...,0.0,"[relate, search, operational, definitions, exa...","[beechen, definition]"
453656,1088718,2728475,weather for avon in,"In English, the name Avon means-the river.The ...",0.0,"[english, name, avon, meansthe, riverthe, name...","[weather, avon]"
453657,1085348,7335165,what county is polson montana in?,Average Teacher Salary in Montana : Montana Te...,0.0,"[average, teacher, salary, montana, montana, t...","[county, polson, montana]"
453658,155056,3966495,do astronauts sleep on their back,"When you’re suffering from a bad back, it can ...",0.0,"[suffer, bad, back, struggle, find, best, matt...","[astronauts, sleep, back]"


In [311]:
validation_data['passage_cleaned']=validation_data.passage.apply(lambda x: preprocessing(x))
validation_data['query_cleaned']=validation_data.queries.apply(lambda x: preprocessing(x))

In [312]:
validation_data

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned
0,988416,4919841,who moses asaga,1 The breech-loading rifle was invented by Cap...,0.0,"[breechloading, rifle, invent, captain, patric...","[moses, asaga]"
1,987720,6039412,where is meriden ct,"CT, or the Corporation Trust Company, is a who...",0.0,"[ct, corporation, trust, company, wholly, own,...","[meriden, ct]"
2,405985,332147,is chatham ct incorporated,CT Angiography of the Coronary Arteries. CT An...,0.0,"[ct, angiography, coronary, arteries, ct, angi...","[chatham, ct, incorporate]"
3,42555,5939644,average salary for primary care sports medicin...,"In its 2011 salary survey, medical staffing fi...",0.0,"[salary, survey, medical, staff, firm, merritt...","[average, salary, primary, care, sport, medici..."
4,1096739,3297735,how many pounds of fat can you lose in a week,"In the calculator below, indicate your average...",0.0,"[calculator, indicate, average, number, drink,...","[many, pound, fat, lose, week]"
...,...,...,...,...,...,...,...
114027,206806,3737250,how a cars manufacturer date,Evaporated milk is milk that is dehydrated unt...,0.0,"[evaporate, milk, milk, dehydrate, water, remo...","[cars, manufacturer, date]"
114028,991590,7301335,where is ridgewood ny located,Msg #346972. If the signing took place in NY a...,0.0,"[msg, sign, take, place, ny, ny, notary, alter...","[ridgewood, ny, locate]"
114029,1087675,1080515,what are deposit solutions banking,Users of Online Banking have always authentica...,0.0,"[users, online, bank, always, authenticate, ga...","[deposit, solutions, bank]"
114030,766808,3188367,what is long term results on radiation seed im...,The term seed also has a general meaning that ...,0.0,"[term, seed, also, general, mean, antedate, ab...","[long, term, result, radiation, seed, implant]"


In [313]:
# train_data.to_pickle('train_data_sample_no_features2.pkl')
# validation_data.to_pickle('validation_data_no_features2.pkl')

# Subtask 1

In [4]:
# train_data = pd.read_pickle('train_data_sample_no_features2.pkl')
# validation_data = pd.read_pickle('validation_data_no_features2.pkl')
train_data = pd.read_pickle('train_data.pkl')
validation_data = pd.read_pickle('validation_data.pkl')

In [260]:
validation_data

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity,bm25,word_mover_distance
0,405985,332147,is chatham ct incorporated,CT Angiography of the Coronary Arteries. CT An...,0.0,"[ct, angiography, coronary, arteries, ct, angi...","[chatham, ct, incorporate]",0.369007,17.481736,1.169729
1,42555,5939644,average salary for primary care sports medicin...,"In its 2011 salary survey, medical staffing fi...",0.0,"[salary, survey, medical, staff, firm, merritt...","[average, salary, primary, care, sport, medici...",0.828698,52.017312,0.841582
2,1096739,3297735,how many pounds of fat can you lose in a week,"In the calculator below, indicate your average...",0.0,"[calculator, indicate, average, number, drink,...","[many, pound, fat, lose, week]",0.770259,43.844760,0.891009
3,622725,4059964,what do decomposers and scavengers,Energy that is not used in an ecosystem is eve...,0.0,"[energy, use, ecosystem, eventually, lose, hea...","[decomposers, scavengers]",0.082339,11.233849,1.337895
4,60677,5652468,calories in one slice ham,Calories per Ingredient. Here are the foods fr...,0.0,"[calories, per, ingredient, foods, food, nutri...","[calories, one, slice, ham]",0.787347,30.288981,0.982390
...,...,...,...,...,...,...,...,...,...,...
54129,1037341,2130295,who is quinn of ny,"Centereach, NY Other Information. 1 Located i...",0.0,"[centereach, ny, information, locate, suffolk,...","[quinn, ny]",0.665690,21.000746,1.016593
54130,352236,6178442,how to cook beets without bleeding,1 Bake the beets on a baking sheet until tende...,0.0,"[bake, beets, bake, sheet, tender, pierce, kni...","[cook, beets, without, bleed]",0.752483,19.215004,1.019188
54131,991590,7301335,where is ridgewood ny located,Msg #346972. If the signing took place in NY a...,0.0,"[msg, sign, take, place, ny, ny, notary, alter...","[ridgewood, ny, locate]",0.278403,18.600281,1.188505
54132,1087675,1080515,what are deposit solutions banking,Users of Online Banking have always authentica...,0.0,"[users, online, bank, always, authenticate, ga...","[deposit, solutions, bank]",0.610685,18.947558,1.055076


In [263]:
validation_data[validation_data['qid'] == 715189].shape

(38, 10)

## BM25

In [208]:
validation_data_no_dup_passages = validation_data.drop_duplicates(subset=['pid'], inplace=False)
validation_data_no_dup_passages

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity,bm25,word_mover_distance
0,405985,332147,is chatham ct incorporated,CT Angiography of the Coronary Arteries. CT An...,0.0,"[ct, angiography, coronary, arteries, ct, angi...","[chatham, ct, incorporate]",0.369007,17.481736,1.169729
1,42555,5939644,average salary for primary care sports medicin...,"In its 2011 salary survey, medical staffing fi...",0.0,"[salary, survey, medical, staff, firm, merritt...","[average, salary, primary, care, sport, medici...",0.828698,52.017312,0.841582
2,1096739,3297735,how many pounds of fat can you lose in a week,"In the calculator below, indicate your average...",0.0,"[calculator, indicate, average, number, drink,...","[many, pound, fat, lose, week]",0.770259,43.844760,0.891009
3,622725,4059964,what do decomposers and scavengers,Energy that is not used in an ecosystem is eve...,0.0,"[energy, use, ecosystem, eventually, lose, hea...","[decomposers, scavengers]",0.082339,11.233849,1.337895
4,60677,5652468,calories in one slice ham,Calories per Ingredient. Here are the foods fr...,0.0,"[calories, per, ingredient, foods, food, nutri...","[calories, one, slice, ham]",0.787347,30.288981,0.982390
...,...,...,...,...,...,...,...,...,...,...
54129,1037341,2130295,who is quinn of ny,"Centereach, NY Other Information. 1 Located i...",0.0,"[centereach, ny, information, locate, suffolk,...","[quinn, ny]",0.665690,21.000746,1.016593
54130,352236,6178442,how to cook beets without bleeding,1 Bake the beets on a baking sheet until tende...,0.0,"[bake, beets, bake, sheet, tender, pierce, kni...","[cook, beets, without, bleed]",0.752483,19.215004,1.019188
54131,991590,7301335,where is ridgewood ny located,Msg #346972. If the signing took place in NY a...,0.0,"[msg, sign, take, place, ny, ny, notary, alter...","[ridgewood, ny, locate]",0.278403,18.600281,1.188505
54132,1087675,1080515,what are deposit solutions banking,Users of Online Banking have always authentica...,0.0,"[users, online, bank, always, authenticate, ga...","[deposit, solutions, bank]",0.610685,18.947558,1.055076


In [190]:
# validation_data_no_dup_passages.head(20)
validation_data_no_dup_passages.shape

(52781, 10)

In [211]:
np.unique(validation_data_no_dup_passages.qid.values).shape

(1065,)

## Inverted_Index

In [194]:
import nltk
passage_cleaned_all = []
for lst in validation_data_no_dup_passages.passage_cleaned.values:
    passage_cleaned_all = passage_cleaned_all + list(set(lst))

In [195]:
inverted_index = nltk.FreqDist(passage_cleaned_all)


FreqDist({'use': 7052, 'one': 5607, 'also': 4981, 'make': 4276, 'may': 4259, 'state': 4096, 'time': 3922, 'get': 3100, 'know': 3084, 'include': 2988, ...})

In [223]:
inverted_index['use']

7052

In [197]:
import pickle
file_to_write = open("inverted_index.pkl", "wb")

pickle.dump(inverted_index, file_to_write)

In [ ]:
# open
# with open('filename.pickle', 'rb') as handle:
#     b = pickle.load(handle)

In [224]:
def get_passage_average_length_and_total_word_occurences_corpus():
    number_of_passages = len(validation_data_no_dup_passages)
    count_total_length = 0
    for idx, row in validation_data_no_dup_passages.iterrows():
        count_total_length += len(row['passage_cleaned'])
    return count_total_length, count_total_length/number_of_passages 


In [225]:
total_word_occurences, avdl = get_passage_average_length_and_total_word_occurences_corpus()


In [226]:
total_word_occurences
avdl

1476383

27.97186487561812

In [227]:
from math import log

k1 = 1.2
k2 = 100
b = 0.75
R = 0
r = 0
N = len(validation_data_no_dup_passages)

def K_cal(dl):
    return k1 * ((1-b) + b * (float(dl)/float(avdl)) )


def BM25_cal(query_tokens, passage_tokens):
    query_length = len(query_tokens)
    query_token_freq_dict = nltk.FreqDist(query_tokens)
    passage_token_freq_dict = nltk.FreqDist(passage_tokens)
    dl = len(passage_tokens)
    K = K_cal(dl)
    score = 0
    for token in query_tokens:
        n = inverted_index[token]
        f = passage_token_freq_dict[token]
        qf = query_token_freq_dict[token]
        first_term = log( ( (r + 0.5) / (R - r + 0.5) ) / ( (n - r + 0.5) / (N - n - R + r + 0.5)) )
        second_term = ((k1 + 1) * f) / (K + f)
        third_term = ((k2+1) * qf) / (k2 + qf)
        score += first_term * second_term * third_term
    return score
    

In [275]:
bm25_cal_dict = {}

for qid in np.unique(validation_data.qid.values):
    bm25_cal_dict[qid] = []
    df_qid = validation_data[validation_data['qid'] == qid]
    for idx, row in df_qid.iterrows():
        query = row['query_cleaned']
        passage = row['passage_cleaned']
        bm25_cal_dict[qid].append(BM25_cal(query, passage))
#         if(row['relevancy']):
#             print(row['relevancy'])
#             print("BM25_cal(query, passage):", BM25_cal(query, passage))        
    

In [276]:
bm25_cal_dict

{2962: [10.51910707910633,
  12.388913966409094,
  11.038895805258974,
  10.661458383586845,
  17.878895219783512,
  11.320095103396081,
  15.845996703192743,
  11.454276708575243,
  10.036174159561476,
  10.75851917191444,
  11.756091312445802,
  11.081868286625973,
  16.514158258735222,
  10.063574899485008,
  11.014554926501376,
  10.318925337501803,
  9.71645787907012,
  13.542670701624832,
  11.670288902861849,
  12.338023969193403,
  11.966878163194368,
  12.02545524793265,
  14.714546075344753,
  9.79446192941091,
  9.871520945268259,
  11.417970732071648,
  9.639686475543623,
  16.317938831206718,
  10.318925337501803,
  11.756091312445802,
  10.382014973644413,
  14.105759791783875,
  17.434204193661387,
  11.935116759412814,
  15.928311493988792,
  12.970126985956258,
  10.384800620731337,
  10.806164759408365,
  14.836519951276529],
 4696: [16.55843640351316,
  16.574172312221968,
  16.428659141860237,
  16.574172312221968,
  16.6715931440895,
  22.48708235407787,
  16.18252

In [250]:
# np.sort(np.array(bm25_cal_dict[2962]))[-100:][::-1]

array([17.87889522, 17.43420419, 16.51415826, 16.31793883, 15.92831149,
       15.8459967 , 14.83651995, 14.71454608, 14.10575979, 13.5426707 ,
       12.97012699, 12.38891397, 12.33802397, 12.02545525, 11.96687816,
       11.93511676, 11.75609131, 11.75609131, 11.6702889 , 11.45427671,
       11.41797073, 11.3200951 , 11.08186829, 11.03889581, 11.01455493,
       10.80616476, 10.75851917, 10.66145838, 10.51910708, 10.38480062,
       10.38201497, 10.31892534, 10.31892534, 10.0635749 , 10.03617416,
        9.87152095,  9.79446193,  9.71645788,  9.63968648])

In [253]:
# results_bm25[2962]

array([ 4, 32, 12, 27, 34,  6, 38, 22, 31, 17, 35,  1, 19, 21, 20, 33, 29,
       10, 18,  7, 25,  5, 11,  2, 14, 37,  9,  3,  0, 36, 30, 28, 15, 13,
        8, 24, 23, 16, 26], dtype=int64)

In [258]:
np.array(bm25_cal_dict[2962])[results_bm25[2962]]

array([17.87889522, 17.43420419, 16.51415826, 16.31793883, 15.92831149,
       15.8459967 , 14.83651995, 14.71454608, 14.10575979, 13.5426707 ,
       12.97012699, 12.38891397, 12.33802397, 12.02545525, 11.96687816,
       11.93511676, 11.75609131, 11.75609131, 11.6702889 , 11.45427671,
       11.41797073, 11.3200951 , 11.08186829, 11.03889581, 11.01455493,
       10.80616476, 10.75851917, 10.66145838, 10.51910708, 10.38480062,
       10.38201497, 10.31892534, 10.31892534, 10.0635749 , 10.03617416,
        9.87152095,  9.79446193,  9.71645788,  9.63968648])

In [265]:
def bm25_ranking(number_of_retrieved, bm25_array):
    result = np.array(bm25_array).argsort()[-number_of_retrieved:][::-1]
    return result

In [266]:
results_bm25 = {}
ranking_k = 100
for qid, lst in bm25_cal_dict.items():
    results_bm25[qid] = bm25_ranking(ranking_k, lst)
    

In [252]:
len(results_bm25)

1065

In [271]:
def average_precision_cal(ranking_list_df):
    ranking_list_df = ranking_list_df.reset_index(drop=True, inplace=False)
    total_relevant_retrieved = 0
    precision_sum = 0
    for idx, row in ranking_list_df.iterrows():
        relevancy = row['relevancy']
        if (relevancy):
#             isRelevant = True
            total_relevant_retrieved += 1
            precision = total_relevant_retrieved / (idx + 1)
            precision_sum += precision
    result = precision_sum / len(ranking_list_df)
    return result

In [272]:
def get_IDCG(ranking_list_df):
    ranking_list_df_sorted = ranking_list_df.sort_values(by=['relevancy'], ascending=False)
#     ranking_list_df_sorted = ranking_list_df.reset_index(drop=True, inplace=False)
    ranking_list_df_sorted = ranking_list_df_sorted.reset_index().reindex(ranking_list_df_sorted.columns, axis=1)
    ideal_discounted_gain_sum = 0
    for idx, row in ranking_list_df_sorted.iterrows():
        index = idx + 1
        relevance_score = row['relevancy']
        gain = 2 ** relevance_score - 1
        discounted_gain = gain / math.log2(index + 1)
        ideal_discounted_gain_sum += discounted_gain
    return ideal_discounted_gain_sum

In [273]:
def get_NDCG(ranking_list_df):
    ranking_list_df = ranking_list_df.reset_index(drop=True, inplace=False)
    discounted_gain_sum = 0
    for idx, row in ranking_list_df.iterrows():
        index = idx + 1
        relevance_score = row['relevancy']
        gain = 2 ** relevance_score - 1
        discounted_gain = gain / math.log2(index + 1)
        discounted_gain_sum += discounted_gain
    IDCG = get_IDCG(ranking_list_df)
    if IDCG == 0:
        return 0
    nDCG = discounted_gain_sum / IDCG
    return nDCG

In [282]:
mean_average_precision = 0
mean_NDCG = 0
for qid, idx_arr in results_bm25.items():
    df_qid = validation_data[validation_data['qid'] == qid]
    ranking_list_df = df_qid.iloc[idx_arr]
    average_precision = average_precision_cal(ranking_list_df)
    NDCG = get_NDCG(ranking_list_df)
    mean_average_precision += average_precision
    mean_NDCG += NDCG
mean_average_precision = mean_average_precision / len(results_bm25)
mean_NDCG = mean_NDCG / len(results_bm25)

In [281]:
mean_average_precision
mean_NDCG

0.007399823694151146

0.39825417807651065

# Subtask 2

In [173]:
# train_data = pd.read_pickle('train_data.pkl')
# validation_data = pd.read_pickle('validation_data.pkl')

## Loading word2vec embedding

In [8]:
# loading pre-trained embeddings, each word is represented as a 300 dimensional vector
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
W2V_PATH="../glove.6B.300d.txt"



In [9]:
import gensim
import os
import shutil
import hashlib
from sys import platform

def getFileLineNums(filename):
    f = open(filename, 'r', encoding='utf-8')
    count = 0
    for line in f:
        count += 1
    return count


def prepend_line(infile, outfile, line):
    with open(infile, 'r') as old:
        with open(outfile, 'w') as new:
            new.write(str(line) + "\n")
            shutil.copyfileobj(old, new)

def prepend_slow(infile, outfile, line):
    with open(infile, 'r', encoding='utf-8') as fin:
        with open(outfile, 'w', encoding='utf-8') as fout:
            fout.write(line + "\n")
            for line in fin:
                fout.write(line)

def load(filename):
    num_lines = getFileLineNums(filename)
    gensim_file = 'glove_model.txt'
    gensim_first_line = "{} {}".format(num_lines, 300)
    # Prepends the line.
    if platform == "linux" or platform == "linux2":
        prepend_line(filename, gensim_file, gensim_first_line)
    else:
        prepend_slow(filename, gensim_file, gensim_first_line)

    model = gensim.models.KeyedVectors.load_word2vec_format(gensim_file)
    return model
model_w2v = load(W2V_PATH)

In [10]:
model_w2v

# Embedding documents and queries

## removing rows that is not defined in the embeddings

### train data

In [11]:
remove_idx = []
for i, row in train_data.iterrows():
    passage = row['passage_cleaned']
    query = row['query_cleaned']
    passage_length = len(passage)
    query_length = len(query)
    if query_length == 0:
        remove_idx.append(i)
        continue
    for j in range(passage_length):
        word = passage[j]
        if not word in model_w2v:
            remove_idx.append(i)
            break
    for k in range(query_length):
        word = query[k]
        if not word in model_w2v:
            remove_idx.append(i)
            break


In [12]:
remove_idx = np.array(remove_idx)
remove_idx = np.unique(remove_idx)

In [13]:
remove_idx.shape

(240131,)

In [14]:
train_data.drop(train_data.index[remove_idx], inplace=True)

In [15]:
train_data.reset_index(drop=True, inplace=True)

### validation data

In [16]:
remove_idx = []
for i, row in validation_data.iterrows():
    passage = row['passage_cleaned']
    query = row['query_cleaned']
    passage_length = len(passage)
    query_length = len(query)
    if query_length == 0:
        remove_idx.append(i)
        continue
    for j in range(passage_length):
        word = passage[j]
        if not word in model_w2v:
            remove_idx.append(i)
            break
    for k in range(query_length):
        word = query[k]
        if not word in model_w2v:
            remove_idx.append(i)
            break


In [17]:
remove_idx = np.array(remove_idx)
remove_idx = np.unique(remove_idx)

In [18]:
validation_data.drop(validation_data.index[remove_idx], inplace=True)

In [19]:
validation_data.reset_index(drop=True, inplace=True)

In [20]:
validation_data

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned
0,405985,332147,is chatham ct incorporated,CT Angiography of the Coronary Arteries. CT An...,0.0,"[ct, angiography, coronary, arteries, ct, angi...","[chatham, ct, incorporate]"
1,42555,5939644,average salary for primary care sports medicin...,"In its 2011 salary survey, medical staffing fi...",0.0,"[salary, survey, medical, staff, firm, merritt...","[average, salary, primary, care, sport, medici..."
2,1096739,3297735,how many pounds of fat can you lose in a week,"In the calculator below, indicate your average...",0.0,"[calculator, indicate, average, number, drink,...","[many, pound, fat, lose, week]"
3,622725,4059964,what do decomposers and scavengers,Energy that is not used in an ecosystem is eve...,0.0,"[energy, use, ecosystem, eventually, lose, hea...","[decomposers, scavengers]"
4,60677,5652468,calories in one slice ham,Calories per Ingredient. Here are the foods fr...,0.0,"[calories, per, ingredient, foods, food, nutri...","[calories, one, slice, ham]"
...,...,...,...,...,...,...,...
54129,1037341,2130295,who is quinn of ny,"Centereach, NY Other Information. 1 Located i...",0.0,"[centereach, ny, information, locate, suffolk,...","[quinn, ny]"
54130,352236,6178442,how to cook beets without bleeding,1 Bake the beets on a baking sheet until tende...,0.0,"[bake, beets, bake, sheet, tender, pierce, kni...","[cook, beets, without, bleed]"
54131,991590,7301335,where is ridgewood ny located,Msg #346972. If the signing took place in NY a...,0.0,"[msg, sign, take, place, ny, ny, notary, alter...","[ridgewood, ny, locate]"
54132,1087675,1080515,what are deposit solutions banking,Users of Online Banking have always authentica...,0.0,"[users, online, bank, always, authenticate, ga...","[deposit, solutions, bank]"


## embedding documents

## embedding quries

## 문제점:
1. 특정 단어가 word embedding에 없는 경우가 있음
-> query_vector 의 모든 entry가 0임

-> cosine similarity 계산할 때 nan 이 나옴

temp solution:
cosine similarity 가 0일떄 결과값에 0 을 넣어줌

## creating training data(adding features)

In [21]:
num_of_features = 3 # cosine_sim, bm25, Word Mover's Distance

In [22]:
x_train = np.zeros((train_data.shape[0], num_of_features))
x_val = np.zeros((validation_data.shape[0], num_of_features))
x_train.shape

(213529, 3)

In [23]:
x_train[0][0]

0.0

In [24]:
def get_document_vector(passage):
    passage_max_length = 150 # document length including padding
    passage_length = len(passage)
    document_word_embedding =np.zeros((passage_max_length,300)) # 64 == padding
    for i in range(passage_length): 
        word = passage[i]
        document_word_embedding[i] = model_w2v[word]
    return document_word_embedding
    

In [25]:
def get_query_vector(query):
    query_max_length = 64 # query length including padding    
    query_length = len(query)
    query_word_embedding =np.zeros((query_max_length,300)) # 64 == padding
    for i in range(query_length): 
        word = query[i]
        query_word_embedding[i] = model_w2v[word]
    return query_word_embedding

### Cosine Simliarity

In [26]:
def cosine_sim_formula(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    if np.isnan(cos_sim):
        cos_sim = 0
    return cos_sim

In [27]:
for i, row in train_data.iterrows():
    query_vector = get_query_vector(row['query_cleaned'])
    passage_vector = get_document_vector(row['passage_cleaned'])
    average_query_vectors = np.mean(query_vector, axis=0)
    average_document_vectors = np.mean(passage_vector, axis=0)
    x_train[i][0] = cosine_sim_formula(average_query_vectors, average_document_vectors)


In [28]:
np.isnan(x_train).any()

False

In [29]:
train_data['co_similarity'] = x_train[:,0]

In [30]:
train_data

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity
0,406386,7869580,is cider vinegar safe,2. It can make your hair shine. Apple cider vi...,0.0,"[make, hair, shine, apple, cider, vinegar, use...","[cider, vinegar, safe]",0.755494
1,1087603,1141888,what are field peas,"Dogs can eat raw or cooked peas, but cooked pe...",0.0,"[dog, eat, raw, cook, peas, cook, peas, softer...","[field, peas]",0.546223
2,115704,931826,david tepper net worth,Videos Earthquake The Comedian Net Worth 2017 ...,0.0,"[videos, earthquake, comedian, net, worth, dig...","[david, tepper, net, worth]",0.575517
3,1096551,8213401,how many valence electrons does an atom of the...,Those elements certainly have more than 0 vale...,0.0,"[elements, certainly, valence, electrons, fact...","[many, valence, electrons, atom, element, neon...",0.799804
4,490802,4649401,russ simmons net worth,Bebe Winans Net Worth. Bebe Winans Net Worth i...,0.0,"[bebe, winans, net, worth, bebe, winans, net, ...","[russ, simmons, net, worth]",0.619965
...,...,...,...,...,...,...,...,...
213524,660803,2204762,what foods and spices trigger vertigo,Foods that should not be included in your diet...,0.0,"[foods, include, diet, difficult, swallow, unp...","[foods, spice, trigger, vertigo]",0.583361
213525,228738,4411837,how does your cervix change during pregnancy,Knowledge about the position and of cervix in ...,0.0,"[knowledge, position, cervix, early, pregnancy...","[cervix, change, pregnancy]",0.732572
213526,1092724,7755456,midland states bank aba number,bank routing number lookup Not looking for the...,0.0,"[bank, rout, number, lookup, look, pnc, bank, ...","[midland, state, bank, aba, number]",0.747702
213527,1093405,4682759,beechen definition,Related searches for Operational Definitions. ...,0.0,"[relate, search, operational, definitions, exa...","[beechen, definition]",0.399225


In [31]:
for i, row in validation_data.iterrows():
    query_vector = get_query_vector(row['query_cleaned'])
    passage_vector = get_document_vector(row['passage_cleaned'])
    average_query_vectors = np.mean(query_vector, axis=0)
    average_document_vectors = np.mean(passage_vector, axis=0)
    x_val[i][0] = cosine_sim_formula(average_query_vectors, average_document_vectors)


In [32]:
validation_data['co_similarity'] = x_val[:,0]

In [33]:
validation_data

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity
0,405985,332147,is chatham ct incorporated,CT Angiography of the Coronary Arteries. CT An...,0.0,"[ct, angiography, coronary, arteries, ct, angi...","[chatham, ct, incorporate]",0.369007
1,42555,5939644,average salary for primary care sports medicin...,"In its 2011 salary survey, medical staffing fi...",0.0,"[salary, survey, medical, staff, firm, merritt...","[average, salary, primary, care, sport, medici...",0.828698
2,1096739,3297735,how many pounds of fat can you lose in a week,"In the calculator below, indicate your average...",0.0,"[calculator, indicate, average, number, drink,...","[many, pound, fat, lose, week]",0.770259
3,622725,4059964,what do decomposers and scavengers,Energy that is not used in an ecosystem is eve...,0.0,"[energy, use, ecosystem, eventually, lose, hea...","[decomposers, scavengers]",0.082339
4,60677,5652468,calories in one slice ham,Calories per Ingredient. Here are the foods fr...,0.0,"[calories, per, ingredient, foods, food, nutri...","[calories, one, slice, ham]",0.787347
...,...,...,...,...,...,...,...,...
54129,1037341,2130295,who is quinn of ny,"Centereach, NY Other Information. 1 Located i...",0.0,"[centereach, ny, information, locate, suffolk,...","[quinn, ny]",0.665690
54130,352236,6178442,how to cook beets without bleeding,1 Bake the beets on a baking sheet until tende...,0.0,"[bake, beets, bake, sheet, tender, pierce, kni...","[cook, beets, without, bleed]",0.752483
54131,991590,7301335,where is ridgewood ny located,Msg #346972. If the signing took place in NY a...,0.0,"[msg, sign, take, place, ny, ny, notary, alter...","[ridgewood, ny, locate]",0.278403
54132,1087675,1080515,what are deposit solutions banking,Users of Online Banking have always authentica...,0.0,"[users, online, bank, always, authenticate, ga...","[deposit, solutions, bank]",0.610685


In [34]:
y_train = train_data.relevancy.values
y_val = validation_data.relevancy.values

### BM25

In [284]:
train_data_no_dup_passages = train_data.drop_duplicates(subset=['pid'], inplace=False)
validation_data_no_dup_passages = validation_data.drop_duplicates(subset=['pid'], inplace=False)

In [289]:
train_data_no_dup_passages.shape
validation_data_no_dup_passages.shape
N_train = train_data_no_dup_passages.shape[0]
N_val = validation_data_no_dup_passages.shape[0]

(200496, 10)

(52781, 10)

In [290]:
def get_passage_average_length(dataframe_no_dup_passages):
    number_of_passages = dataframe_no_dup_passages.shape[0]
    count_total_length = 0
    for idx, row in dataframe_no_dup_passages.iterrows():
        count_total_length += len(row['passage_cleaned'])
    return count_total_length/number_of_passages 


In [291]:
# total_word_occurences, avdl = get_passage_average_length_and_total_word_occurences_corpus()
avdl_train = get_passage_average_length(train_data_no_dup_passages)
avdl_val = get_passage_average_length(validation_data_no_dup_passages)

In [297]:
train_data_no_dup_passages.passage_cleaned.values

array([list(['make', 'hair', 'shine', 'apple', 'cider', 'vinegar', 'use', 'rinse', 'hair', 'shampoo', 'boost', 'hair', 'body', 'shine', 'recommend', 'recycle', 'old', 'shampoo', 'bottle', 'fill', 'tablespoon', 'apple', 'cider', 'vinegar', 'cup', 'cold', 'water', 'apple', 'cider', 'vinegar', 'aid', 'weight', 'loss', 'daily', 'weight', 'management', 'add', 'teaspoons', 'apple', 'cider', 'vinegar', 'ounces', 'water', 'concoction', 'sip', 'throughout', 'day']),
       list(['dog', 'eat', 'raw', 'cook', 'peas', 'cook', 'peas', 'softer', 'easier', 'digest', 'peas', 'also', 'promote', 'bowel', 'movements', 'feed', 'dog', 'large', 'quantity', 'peas', 'cause', 'diarrhea', 'gastrointestinal', 'problems', 'experts', 'recommend', 'give', 'peas', 'vegetables', 'part', 'balance', 'diet', 'nt', 'make', 'percent', 'dog', 'diet']),
       list(['videos', 'earthquake', 'comedian', 'net', 'worth', 'digital', 'earthquake', 'net', 'worth', 'earthquake', 'net', 'worth', 'earthquake', 'american', 'actor', 'v

In [305]:
train_passages_list = list(train_data_no_dup_passages.passage_cleaned.values)


In [306]:
import itertools
passage_cleaned_all = list(itertools.chain.from_iterable(train_passages_list))

In [307]:
len(passage_cleaned_all)

5607350

In [308]:
inverted_index_train = nltk.FreqDist(passage_cleaned_all)
 

In [309]:
inverted_index_train

FreqDist({'use': 33806, 'one': 25338, 'time': 21571, 'state': 21329, 'may': 20837, 'also': 20195, 'make': 19396, 'number': 18925, 'cause': 17987, 'county': 17296, ...})

In [310]:
import pickle
file_to_write = open("inverted_index_train.pkl", "wb")

pickle.dump(inverted_index_train, file_to_write)

In [311]:
from math import log

k1 = 1.2
k2 = 100
b = 0.75
R = 0
r = 0
# N = len(validation_data_no_dup_passages)

def K_cal(dl, avdl):
    return k1 * ((1-b) + b * (float(dl)/float(avdl)) )


def BM25_cal_for_preprocessed_words(query_tokens, passage_tokens, N, avdl, inverted_index):
    query_length = len(query_tokens)
    query_token_freq_dict = nltk.FreqDist(query_tokens)
    passage_token_freq_dict = nltk.FreqDist(passage_tokens)
    dl = len(passage_tokens)
    K = K_cal(dl, avdl)
    score = 0
    for token in query_tokens:
        n = inverted_index[token]
        f = passage_token_freq_dict[token]
        qf = query_token_freq_dict[token]
        first_term = log( ( (r + 0.5) / (R - r + 0.5) ) / ( (n - r + 0.5) / (N - n - R + r + 0.5)) )
        second_term = ((k1 + 1) * f) / (K + f)
        third_term = ((k2+1) * qf) / (k2 + qf)
        score += first_term * second_term * third_term
    return score
    

In [313]:
for idx, row in train_data.iterrows():
    query_cleaned = row['query_cleaned']
    passage_cleaned = row['passage_cleaned']
    bm25_score = BM25_cal_for_preprocessed_words(query_cleaned, passage_cleaned, N_train, avdl_train, inverted_index_train)
    x_train[idx][1] = bm25_score

In [314]:
for idx, row in validation_data.iterrows():
    query_cleaned = row['query_cleaned']
    passage_cleaned = row['passage_cleaned']
    bm25_score = BM25_cal_for_preprocessed_words(query_cleaned, passage_cleaned, N_val, avdl_val, inverted_index)
    x_val[idx][1] = bm25_score

In [315]:
train_data['bm25'] = x_train[:,1]
validation_data['bm25'] = x_val[:,1]

In [316]:
train_data

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity,bm25,word_mover_distance
0,406386,7869580,is cider vinegar safe,2. It can make your hair shine. Apple cider vi...,0.0,"[make, hair, shine, apple, cider, vinegar, use...","[cider, vinegar, safe]",0.755494,16.979707,0.990366
1,1087603,1141888,what are field peas,"Dogs can eat raw or cooked peas, but cooked pe...",0.0,"[dog, eat, raw, cook, peas, cook, peas, softer...","[field, peas]",0.546223,11.203316,1.097962
2,115704,931826,david tepper net worth,Videos Earthquake The Comedian Net Worth 2017 ...,0.0,"[videos, earthquake, comedian, net, worth, dig...","[david, tepper, net, worth]",0.575517,13.763724,0.850333
3,1096551,8213401,how many valence electrons does an atom of the...,Those elements certainly have more than 0 vale...,0.0,"[elements, certainly, valence, electrons, fact...","[many, valence, electrons, atom, element, neon...",0.799804,23.575800,0.828285
4,490802,4649401,russ simmons net worth,Bebe Winans Net Worth. Bebe Winans Net Worth i...,0.0,"[bebe, winans, net, worth, bebe, winans, net, ...","[russ, simmons, net, worth]",0.619965,13.137259,0.896055
...,...,...,...,...,...,...,...,...,...,...
213524,660803,2204762,what foods and spices trigger vertigo,Foods that should not be included in your diet...,0.0,"[foods, include, diet, difficult, swallow, unp...","[foods, spice, trigger, vertigo]",0.583361,12.523914,1.105572
213525,228738,4411837,how does your cervix change during pregnancy,Knowledge about the position and of cervix in ...,0.0,"[knowledge, position, cervix, early, pregnancy...","[cervix, change, pregnancy]",0.732572,16.363526,0.948814
213526,1092724,7755456,midland states bank aba number,bank routing number lookup Not looking for the...,0.0,"[bank, rout, number, lookup, look, pnc, bank, ...","[midland, state, bank, aba, number]",0.747702,17.261806,0.779405
213527,1093405,4682759,beechen definition,Related searches for Operational Definitions. ...,0.0,"[relate, search, operational, definitions, exa...","[beechen, definition]",0.399225,5.876990,1.033844


## Word Mover's Distance

In [44]:
x_train

array([[ 0.75549353, 48.3316421 ,  0.        ],
       [ 0.54622331, 25.28191511,  0.        ],
       [ 0.57551721, 50.5178451 ,  0.        ],
       ...,
       [ 0.74770237, 69.68721785,  0.        ],
       [ 0.39922497, 26.36072877,  0.        ],
       [ 0.25682087, 24.01551805,  0.        ]])

In [45]:
for i, row in train_data.iterrows():
    query = row['query_cleaned']
    passage = row['passage_cleaned']
#     print(model_w2v.wmdistance(query, passage))
    x_train[i][2] = model_w2v.wmdistance(query, passage)


In [46]:
x_train

array([[ 0.75549353, 48.3316421 ,  0.9903657 ],
       [ 0.54622331, 25.28191511,  1.09796191],
       [ 0.57551721, 50.5178451 ,  0.8503327 ],
       ...,
       [ 0.74770237, 69.68721785,  0.77940468],
       [ 0.39922497, 26.36072877,  1.03384396],
       [ 0.25682087, 24.01551805,  1.22079097]])

In [47]:
train_data['word_mover_distance'] = x_train[:,2]

In [48]:
train_data[train_data['word_mover_distance'] == np.inf]

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity,bm25,word_mover_distance


In [49]:
for i, row in validation_data.iterrows():
    query = row['query_cleaned']
    passage = row['passage_cleaned']
#     print(model_w2v.wmdistance(query, passage))
    x_val[i][2] = model_w2v.wmdistance(query, passage)


In [50]:
validation_data['word_mover_distance'] = x_val[:,2]

In [317]:
validation_data

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity,bm25,word_mover_distance
0,405985,332147,is chatham ct incorporated,CT Angiography of the Coronary Arteries. CT An...,0.0,"[ct, angiography, coronary, arteries, ct, angi...","[chatham, ct, incorporate]",0.369007,9.115305,1.169729
1,42555,5939644,average salary for primary care sports medicin...,"In its 2011 salary survey, medical staffing fi...",0.0,"[salary, survey, medical, staff, firm, merritt...","[average, salary, primary, care, sport, medici...",0.828698,19.120867,0.841582
2,1096739,3297735,how many pounds of fat can you lose in a week,"In the calculator below, indicate your average...",0.0,"[calculator, indicate, average, number, drink,...","[many, pound, fat, lose, week]",0.770259,18.021866,0.891009
3,622725,4059964,what do decomposers and scavengers,Energy that is not used in an ecosystem is eve...,0.0,"[energy, use, ecosystem, eventually, lose, hea...","[decomposers, scavengers]",0.082339,7.240845,1.337895
4,60677,5652468,calories in one slice ham,Calories per Ingredient. Here are the foods fr...,0.0,"[calories, per, ingredient, foods, food, nutri...","[calories, one, slice, ham]",0.787347,13.094935,0.982390
...,...,...,...,...,...,...,...,...,...,...
54129,1037341,2130295,who is quinn of ny,"Centereach, NY Other Information. 1 Located i...",0.0,"[centereach, ny, information, locate, suffolk,...","[quinn, ny]",0.665690,9.996924,1.016593
54130,352236,6178442,how to cook beets without bleeding,1 Bake the beets on a baking sheet until tende...,0.0,"[bake, beets, bake, sheet, tender, pierce, kni...","[cook, beets, without, bleed]",0.752483,11.389883,1.019188
54131,991590,7301335,where is ridgewood ny located,Msg #346972. If the signing took place in NY a...,0.0,"[msg, sign, take, place, ny, ny, notary, alter...","[ridgewood, ny, locate]",0.278403,8.854238,1.188505
54132,1087675,1080515,what are deposit solutions banking,Users of Online Banking have always authentica...,0.0,"[users, online, bank, always, authenticate, ga...","[deposit, solutions, bank]",0.610685,7.125731,1.055076


## Saving/loading pickle

In [318]:
# train_data.to_pickle('train_data.pkl')
# validation_data.to_pickle('validation_data.pkl')

## Load

In [ ]:
train_data = pd.read_pickle('train_data.pkl')
validation_data = pd.read_pickle('validation_data.pkl')
num_of_features = 3

In [ ]:
x_train = np.zeros((train_data.shape[0], num_of_features))

In [ ]:
x_train = train_data[['co_similarity', 'bm25', 'word_mover_distance']].values

In [ ]:
x_train.shape

In [ ]:
y_train = train_data['relevancy'].values

In [ ]:
x_val = np.zeros((validation_data.shape[0], num_of_features))

In [ ]:
x_val = validation_data[['co_similarity', 'bm25', 'word_mover_distance']].values

In [ ]:
x_val.shape

In [ ]:
y_val = validation_data['relevancy'].values
y_val

## Convert data into metrics form

In [ ]:
x_train.shape

In [ ]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
x_val = x_val.reshape(x_val.shape[0],x_val.shape[1], 1)
x_train.shape
x_val.shape

In [ ]:
# x_train = x_train.squeeze(2)
# x_val = x_val.squeeze(2)
# x_train.shape

## Logisitc Regression

## 현재 문제 
1. 딥러닝 과제처럼, 한 data씩 처리 할건지, 아니면 인터넷 예제처럼 모든 value를 metrics 에 넣어서 한번에 처리할건지
2. 딥러닝 과제처럼 처리했을 경우, gradient descent function은 무엇인지, 인터넷 예제처럼 했을 때도, 인터넷에 있는 gradient descent function 이 어떻게 derive 됬는지 알기

A: 결국엔 둘다 똑같은 방법인데, metrics 로 처리하는 경우는 한번에 모든 데이터를 처리하는 경우이고, 딥러닝 과제는 한 data sampling 을 하나씩 처리하는 방법이다. 결국엔 둘 다 똑같다.

In [ ]:
class LogisticRegression:

    def __init__(self):
        self.lr = 0.01
        self.epoch = 50
        self.weights = None
        self.bias = None
        self.loss_train = None
        self.loss_val = None

        self.acc_train = None
        self.train_correct = None
        self.val_correct = None
        
        self.losslist_train = []
        self.losslist_val = []
        self.acclist_train = []
        self.acclist_val = []
    

        
    def fit(self, trainxs, trainys, trainxs_val, trainys_val):
        n_samples, n_features, _ = trainxs.shape
        n_samples_val = trainxs_val.shape[0]

        # init parameters
#         self.weights = np.zeros((1, n_features))
        self.weights = np.random.normal(0,1,(1,n_features))#np.random.randn(1,n_features)#np.zeros((1, n_features))
        self.bias = 0
        
        # gradient descent
        for e in range(self.epoch):
            idx = self.shuffleIdx(trainxs.shape[0])
            X = trainxs[idx]
            Y = trainys[idx]
            
            
            self.loss_train = 0
            self.loss_val = 0

            self.acc_train = 0
            self.train_correct = 0
            self.val_correct = 0
            total_weight_gradient1 = 0
            total_weight_gradient2 = 0
            total_weight_gradient3 = 0
            
            total_bias_gradient = 0
            
            for i in range(trainxs.shape[0]):
                x = X[i]
                y = Y[i]
                # approximate output variable (y) with linear combination of weights and x, plus bias
                linear_equation = np.dot(self.weights, x)[0][0] + self.bias
                # apply sigmoid function
                prediction = self.sigmoid(linear_equation)
                if prediction >= 0.5:
                    yprime = 1
                else:
                    yprime = 0

                if yprime == y:
                    self.train_correct += 1

                self.loss_train += self.loss_function(y, prediction)
#                 print("x[2]:", x[2])
                # compute gradients
                dw1 = (prediction - y) * x[0]  #derivative w.r.t weight1
                dw2 = (prediction - y) * x[1] #derivative w.r.t weight2
                dw3 = (prediction - y) * x[2] #derivative w.r.t weight3
                
                db = prediction - y  #derivative w.r.t bias
                    
                    
                total_weight_gradient1 += dw1
                total_weight_gradient2 += dw2
                total_weight_gradient3 += dw3
                
                total_bias_gradient += db
               
                 # COMPUTING LOSS AND ACCURACY OF VALIDATION SET
                if (i < trainxs_val.shape[0]):
                    val_x = trainxs_val[i]
                    val_y = trainys_val[i]
                    linear_equation = np.dot(self.weights, val_x)[0][0] + self.bias
                    prediction = self.sigmoid(linear_equation)

                    if prediction  >= 0.5:
                        yprime = 1
                    else:
                        yprime = 0

                    if yprime == val_y:
                        self.val_correct += 1

                    self.loss_val += ((val_y - prediction)**2)/2

                    
            # COMPUTING THE AVERAGE OF GRADIENTS FOR EACH EPOCH SINCE WE ARE DOING        
            # FULL-BATCH GRADIENT DESCEND AND UPDATING THE PARAMETERS AFTER EACH EPOCH
            total_weight_gradient1 = total_weight_gradient1 / n_samples
            total_weight_gradient2 = total_weight_gradient2 / n_samples
            total_weight_gradient3 = total_weight_gradient3 / n_samples
            print(total_weight_gradient1, total_weight_gradient2, total_weight_gradient3)
            
            total_bias_gradient = total_bias_gradient / n_samples

            # update parameters
#             print(total_weight_gradient1,total_weight_gradient2,total_weight_gradient3)
            self.weights[0][0] -= self.lr * total_weight_gradient1
            self.weights[0][1] -= self.lr * total_weight_gradient2
            self.weights[0][2] -= self.lr * total_weight_gradient3
            
            self.bias -= self.lr * total_bias_gradient
            
            self.loss_train = self.loss_train/n_samples
            self.losslist_train.append(self.loss_train)

            self.loss_val = self.loss_val/(n_samples_val)
            self.losslist_val.append(self.loss_val)

            self.train_correct = self.train_correct/n_samples
            self.acclist_train.append(self.train_correct)

            self.val_correct = self.val_correct/(n_samples_val)
            self.acclist_val.append(self.val_correct)
            
            print("- Loss on Training / Validation Data at Epoch {}: {:.4f} / {:.4f}".format(e+1, self.loss_train, self.loss_val)) 
            print("- Accuracy on Training / Validation Data at Epoch {}: {} / {}\n".format(e+1, self.train_correct, self.val_correct))

            
    def shuffleIdx(self, n):
        rng = default_rng()
        rand_idx = rng.permutation(n)
        return rand_idx

    def predict(self, X):
#         linear_equation = np.dot(X, self.weights) + self.bias
        y_predicted_cls = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            x = X[i]
            linear_equation = np.dot(self.weights, x)[0][0] + self.bias
            y_predicted = self.sigmoid(linear_equation)
#             print("y_predicted:", y_predicted)
            y_predicted_cls[i] = y_predicted
#             print(y_predicted)
#             if y_predicted > 0.5:
#                 y_predicted_cls[i] = 1
#             else:
#                 y_predicted_cls[i] = 0
#             y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return y_predicted_cls

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def loss_function(self, y, prediction):
        prediction = np.clip(prediction,1e-3,1-1e-3)
        ls = -log((1 - prediction)**(1 - y)) - log(prediction**y)
        return ls

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train, x_val, y_val)
# predictions = regressor.predict(xtest)


In [59]:
lr.bias

-0.036969407128965834

In [60]:
lr.weights.shape

(1, 3)

In [61]:
lr.predict(x_val)

array([0.110042  , 0.00114944, 0.00377807, ..., 0.0814134 , 0.12911318,
       0.09345796])

In [65]:
x_val.shape

(54134, 3, 1)

In [82]:
unique_qids_val = np.unique(validation_data.qid.values)
mean_average_precision = 0
mean_NDCG = 0
for qid in unique_qids_val:
    df_qid = validation_data[validation_data['qid'] == qid]
    test = np.zeros((len(df_qid), x_val.shape[1], 1))
    test = df_qid[['co_similarity', 'bm25', 'word_mover_distance']].values
    test = test.reshape(test.shape[0],test.shape[1], 1)
    score = lr.predict(test)
    df_qid['score'] = score
    df_qid_sorted = df_qid.sort_values(by=['score'], ascending=False)
    average_precision = average_precision_cal(df_qid_sorted)
    NDCG = get_NDCG(df_qid_sorted)
#     print(average_precision, NDCG)
    mean_average_precision += average_precision
    mean_NDCG += NDCG
mean_average_precision = mean_average_precision / x_val.shape[0]
mean_NDCG = mean_NDCG / x_val.shape[0]

C:\Users\Kang\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [83]:
mean_average_precision
mean_NDCG

1.934795392541659e-05

0.0022692682494346145

In [139]:
validation_data_lr = validation_data.copy(deep=True)

In [145]:
validation_data_lr['score'] = lr.predict(x_val)
validation_data_lr


,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity,bm25,word_mover_distance,score
0,467612,7552919,oasis hospital contact number,Contact Us Contact Us If you are unable to cal...,1.0,"[contact, us, contact, us, unable, call, oasis...","[oasis, hospital, contact, number]",0.683691,24.411400,1.050934,0.892629
1,1046952,7318074,who played luke on general hospital,It was in the oral and facial unit and it was ...,0.0,"[oral, facial, unit, dentists, room, set, like...","[play, luke, general, hospital]",0.637770,17.520037,1.099775,0.834025
2,1062190,5300247,why did bob marley die of cancer,exists and is an alternate of . Napoleon Bonap...,0.0,"[exist, alternate, napoleon, bonaparte, die, b...","[bob, marley, die, cancer]",0.572388,25.229735,0.938711,0.892457
3,946825,7963463,when does la liga espanola start,Traditional new La Liga season starts on the t...,1.0,"[traditional, new, la, liga, season, start, th...","[la, liga, espanola, start]",0.488907,26.889117,1.041251,0.902010
4,412982,6020416,is indirect labor an overhead cost,Ans: d Response A: Materials requisition slips...,0.0,"[ans, response, materials, requisition, slip, ...","[indirect, labor, overhead, cost]",0.709268,27.235843,1.016331,0.910881
...,...,...,...,...,...,...,...,...,...,...,...
1214,922335,7811374,what was the reason for the snake war,The Snake War has been widely forgotten in Uni...,1.0,"[snake, war, widely, forget, unite, state, his...","[reason, snake, war]",0.775045,26.856588,0.991801,0.910384
1215,341039,7330977,how tall is kate mckinnon,The list of complete Kate McKinnon body measur...,1.0,"[list, complete, kate, mckinnon, body, measure...","[tall, kate, mckinnon]",0.379143,15.984450,1.173580,0.805003
1216,685591,7471912,what is a google brand account?,Hello all Google brand account is that account...,1.0,"[hello, google, brand, account, account, faceb...","[google, brand, account]",0.893495,39.647229,0.657675,0.961808
1217,1097885,7229072,how long nicotine stay in your saliva,Addiction. Nicotine from cigarettes is as addi...,0.0,"[addiction, nicotine, cigarettes, addictive, h...","[long, nicotine, stay, saliva]",0.778365,14.248260,0.933692,0.803246


In [148]:
lr_ranked_list_df = validation_data_lr.sort_values(by=['score'], ascending=False)
lr_ranked_list_df = lr_ranked_list_df.reset_index().reindex(lr_ranked_list_df.columns, axis=1)
lr_ranked_list_df 

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity,bm25,word_mover_distance,score
0,1007691,7251254,"when allocating service department costs, the ...",Direct method allocates each service departmen...,1.0,"[direct, method, allocate, service, department...","[allocate, service, department, cost, method, ...",0.958384,189.441988,0.445933,0.999999
1,1089945,7079883,the __________ test is a quick and dirty test ...,• The Smell Test is familiar ground in most bu...,1.0,"[smell, test, familiar, grind, businesses, goo...","[__________, test, quick, dirty, test, decide,...",0.954141,151.095160,0.468387,0.999986
2,1085764,7116260,what constitutional clause prohibits a state f...,What constitutional clause prohibits a State f...,1.0,"[constitutional, clause, prohibit, state, draw...","[constitutional, clause, prohibit, state, draw...",0.892600,124.247251,0.580840,0.999906
3,8701,7990420,_____ is the ability of cardiac pacemaker cell...,A skeletal muscle contracts only after it is s...,1.0,"[skeletal, muscle, contract, stimulate, nerve,...","[_____, ability, cardiac, pacemaker, cells, sp...",0.948867,109.809304,0.417478,0.999735
4,1093920,7711895,is the process in which we detect physical ene...,Sensation is the process by which we detect ph...,1.0,"[sensation, process, detect, physical, energy,...","[process, detect, physical, energy, environmen...",0.862633,95.906577,0.726514,0.999295
...,...,...,...,...,...,...,...,...,...,...,...
1214,1087566,7100934,what are hcp,"HCP, Inc. is a real estate investment trust wh...",1.0,"[hcp, inc, real, estate, investment, trust, in...",[hcp],-0.010663,7.877156,1.394468,0.673517
1215,714672,7480666,what is an fmo,Florida Marketing Organization has a dedicated...,1.0,"[florida, market, organization, dedicate, focu...",[fmo],-0.161294,6.973536,1.450475,0.648188
1216,293992,2790193,how many product lines does coca cola have,COCA-COLA ON SOCIAL. Coca-Cola Great Britain i...,1.0,"[cocacola, social, cocacola, great, britain, r...","[many, product, line, coca, cola]",0.675006,0.000000,1.120419,0.593006
1217,1029030,7792577,what is utah law on non compete,There are limited situations where a reasonabl...,0.0,"[limit, situations, reasonable, noncompete, ag...","[utah, law, non, compete]",0.620559,0.000000,1.209333,0.591462


In [152]:
average_precision_cal(lr_ranked_list_df)

0.397661437204143

In [153]:
get_NDCG(lr_ranked_list_df)

0.9581420368748004

In [84]:
from sklearn.linear_model import LogisticRegression as Logistic_sklearn

In [85]:
lr_sklearn = Logistic_sklearn()
lr_sklearn.fit(x_train.squeeze(2), y_train)

LogisticRegression()

In [86]:
pred_sklearn = lr_sklearn.predict(x_val.squeeze(2))

In [87]:
score = lr_sklearn.score(x_val.squeeze(2), y_val)
score

0.9882883215723944

# Subtask 3


In [88]:
import pandas as pd
import numpy as np
from xgboost import DMatrix,train



In [140]:
train_data = pd.read_pickle('train_data.pkl')
validation_data = pd.read_pickle('validation_data.pkl')
num_of_features = 3

## Preprocessing the data into the form which XGboost can take

In [141]:
num_of_features = 3

## train data

In [142]:
train_data = train_data.sort_values(by=['qid'], ascending=False)
train_data


,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity,bm25,word_mover_distance
200977,1102400,629910,why do bears hibernate,"Along with the brown bear, it is one of only t...",0.0,"[along, brown, bear, one, two, eight, modern, ...","[bear, hibernate]",0.429928,25.512932,1.114467
133277,1102400,4137507,why do bears hibernate,Rating Newest Oldest. Best Answer: The polar b...,0.0,"[rat, newest, oldest, best, answer, polar, bea...","[bear, hibernate]",0.582447,25.950381,1.040456
14044,1102400,8283394,why do bears hibernate,Use hibernation when you know that you won't u...,0.0,"[use, hibernation, know, wo, nt, use, laptop, ...","[bear, hibernate]",0.272962,18.464533,1.256130
133339,1102400,2505470,why do bears hibernate,Although polar bears (Ursus maritimus) are und...,0.0,"[although, polar, bear, ursus, maritimus, unde...","[bear, hibernate]",0.511174,22.831432,1.155045
180834,1102400,5714169,why do bears hibernate,Black bears are an important and natural compo...,0.0,"[black, bear, important, natural, component, n...","[bear, hibernate]",0.364844,26.038796,1.116584
...,...,...,...,...,...,...,...,...,...,...
158126,2,2831927,Androgen receptor define,Decreased Aggression: One of the most importan...,0.0,"[decrease, aggression, one, important, dog, ne...","[androgen, receptor, define]",0.377254,21.791943,1.221602
81965,2,3215164,Androgen receptor define,An agonist is a chemical capable of binding to...,0.0,"[agonist, chemical, capable, bind, receptor, n...","[androgen, receptor, define]",0.779287,24.483498,0.998807
85516,2,7231072,Androgen receptor define,Rate of the frog heart beat before and after p...,0.0,"[rate, frog, heart, beat, pilocarpine, atropin...","[androgen, receptor, define]",0.694875,23.146895,1.072034
157744,2,704940,Androgen receptor define,Steroid hormones pass through the cell membran...,0.0,"[steroid, hormones, pass, cell, membrane, bind...","[androgen, receptor, define]",0.708429,23.509975,1.073402


In [143]:
lambdaMart_x_train = train_data[['co_similarity', 'bm25', 'word_mover_distance']].values
lambdaMart_x_train

array([[ 0.42992772, 25.51293239,  1.11446663],
       [ 0.58244707, 25.95038068,  1.04045587],
       [ 0.27296237, 18.46453279,  1.25612974],
       ...,
       [ 0.69487517, 23.14689508,  1.07203353],
       [ 0.70842868, 23.50997488,  1.07340152],
       [ 0.64576083, 36.3284463 ,  1.11704851]])

In [144]:
lambdaMart_y_train = train_data.relevancy.values

In [145]:
query_count_dict_train = train_data['qid'].value_counts().to_dict()

In [146]:
query_count_dict_train

{1034446: 88,
 845719: 86,
 1037689: 85,
 1084555: 85,
 330640: 83,
 953445: 83,
 495018: 82,
 10157: 81,
 947466: 80,
 848478: 80,
 998062: 80,
 811266: 80,
 445908: 79,
 1089674: 79,
 503401: 79,
 1091337: 78,
 1091473: 78,
 933652: 77,
 885986: 77,
 551309: 77,
 762558: 77,
 420365: 76,
 1088875: 76,
 182081: 76,
 1032198: 76,
 569689: 76,
 1087870: 76,
 75717: 76,
 172062: 76,
 611366: 76,
 1092715: 75,
 455862: 75,
 1089501: 75,
 503390: 75,
 1084038: 75,
 1017476: 75,
 323798: 75,
 550609: 75,
 1078187: 74,
 1010615: 74,
 506438: 74,
 92542: 74,
 841961: 74,
 1100581: 74,
 632825: 73,
 1053253: 73,
 914845: 73,
 604619: 73,
 667373: 73,
 1083500: 73,
 409694: 73,
 986733: 73,
 503381: 73,
 616415: 73,
 154301: 73,
 605363: 73,
 19457: 73,
 832188: 73,
 81649: 73,
 88577: 73,
 1096476: 72,
 1033443: 72,
 406386: 72,
 56188: 72,
 920717: 72,
 816893: 72,
 1099831: 72,
 878959: 72,
 345350: 72,
 944451: 72,
 138640: 72,
 1084276: 72,
 744092: 72,
 1096454: 71,
 59392: 71,
 1007606: 

In [147]:
len(query_count_dict_train)

4194

In [148]:
dgroup_train = []
dgroup_train

[]

In [149]:
checker = []
for idx, row in train_data.iterrows():
    query = row['qid']
    if query in checker:
        continue
    checker.append(query)
    group_entry_count = query_count_dict_train[query]
    dgroup_train.append(group_entry_count)

In [150]:
# dgroup_train = np.array(dgroup_train)

In [151]:
dgroup_train

[63,
 53,
 64,
 62,
 48,
 56,
 57,
 54,
 50,
 57,
 57,
 50,
 40,
 48,
 48,
 54,
 62,
 50,
 65,
 44,
 47,
 54,
 39,
 44,
 49,
 54,
 45,
 59,
 39,
 66,
 55,
 53,
 47,
 58,
 53,
 51,
 61,
 49,
 49,
 51,
 15,
 44,
 51,
 50,
 59,
 39,
 61,
 54,
 46,
 61,
 48,
 49,
 52,
 22,
 39,
 48,
 46,
 52,
 56,
 55,
 58,
 43,
 44,
 74,
 59,
 48,
 40,
 51,
 60,
 51,
 50,
 49,
 58,
 45,
 43,
 60,
 43,
 45,
 46,
 53,
 58,
 43,
 22,
 26,
 31,
 65,
 46,
 44,
 47,
 63,
 57,
 53,
 37,
 47,
 66,
 49,
 58,
 32,
 65,
 55,
 59,
 72,
 48,
 48,
 56,
 57,
 35,
 56,
 52,
 70,
 50,
 68,
 39,
 56,
 50,
 49,
 47,
 44,
 43,
 56,
 55,
 43,
 57,
 55,
 57,
 20,
 47,
 56,
 58,
 53,
 57,
 53,
 46,
 55,
 68,
 43,
 24,
 61,
 68,
 50,
 44,
 43,
 61,
 49,
 56,
 57,
 50,
 68,
 50,
 65,
 57,
 68,
 68,
 43,
 34,
 60,
 30,
 63,
 54,
 53,
 54,
 58,
 68,
 54,
 48,
 47,
 51,
 45,
 64,
 47,
 55,
 41,
 54,
 58,
 64,
 61,
 60,
 47,
 38,
 49,
 49,
 54,
 54,
 52,
 48,
 41,
 57,
 63,
 55,
 64,
 65,
 45,
 52,
 50,
 45,
 66,
 66,
 44,
 62,
 63,


In [152]:
np.sum(dgroup_train)

213529

## validation data

In [153]:
validation_data = validation_data.sort_values(by=['qid'], ascending=False)
validation_data

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity,bm25,word_mover_distance
12997,1102335,8456194,why do people buy cars,THE COLOUR RED. RED is the colour of LOVE and ...,0.0,"[colour, red, red, colour, love, use, profusel...","[people, buy, cars]",0.660503,33.701546,1.086605
39497,1102335,2344898,why do people buy cars,I been wanting to buy a camshaft and on almost...,0.0,"[want, buy, camshaft, almost, ones, see, say, ...","[people, buy, cars]",0.716632,23.299768,1.064731
5592,1102335,4764512,why do people buy cars,"But for many people, cars are much more than t...",0.0,"[many, people, cars, much, work, art, technolo...","[people, buy, cars]",0.811969,31.370655,0.917742
15427,1102335,5114117,why do people buy cars,That’s why if you plan to freeze ham for more ...,0.0,"[plan, freeze, ham, months, suggest, buy, unco...","[people, buy, cars]",0.658038,31.370655,1.005135
53349,1102335,5953753,why do people buy cars,Bargaining or haggling is a tradition in Turke...,0.0,"[bargain, haggle, tradition, turkey, many, cou...","[people, buy, cars]",0.767158,28.044409,0.983758
...,...,...,...,...,...,...,...,...,...,...
847,2962,2694605,Explain what a bone scan is and what it is use...,Removable drives can always be scanned during ...,0.0,"[removable, drive, always, scan, quick, scan, ...","[explain, bone, scan, use]",0.723047,21.468425,0.944758
19557,2962,2026799,Explain what a bone scan is and what it is use...,A bone scan is a test that can find damage to ...,0.0,"[bone, scan, test, find, damage, bone, find, c...","[explain, bone, scan, use]",0.839605,35.437036,0.922435
13767,2962,6541107,Explain what a bone scan is and what it is use...,If your doctor wants you to have a scan earlie...,0.0,"[doctor, want, scan, earlier, weeks, pregnancy...","[explain, bone, scan, use]",0.739907,19.518334,0.978086
16949,2962,478209,Explain what a bone scan is and what it is use...,scan with a second program after scanning with...,0.0,"[scan, second, program, scan, first, antivirus...","[explain, bone, scan, use]",0.662066,30.566273,1.001465


In [154]:
lambdaMart_x_val = validation_data[['co_similarity', 'bm25', 'word_mover_distance']].values
lambdaMart_x_val

array([[ 0.66050277, 33.70154619,  1.08660517],
       [ 0.71663218, 23.2997681 ,  1.06473102],
       [ 0.81196944, 31.37065491,  0.91774246],
       ...,
       [ 0.73990656, 19.51833397,  0.97808564],
       [ 0.662066  , 30.56627311,  1.00146466],
       [ 0.64282122, 18.89655031,  1.02725494]])

In [155]:
lambdaMart_y_val = validation_data.relevancy.values

In [156]:
query_count_dict_val = validation_data['qid'].value_counts().to_dict()

In [157]:
dgroup_val = []
dgroup_val

[]

In [158]:
checker = []
for idx, row in validation_data.iterrows():
    query = row['qid']
    if query in checker:
        continue
    checker.append(query)
    group_entry_count = query_count_dict_val[query]
    dgroup_val.append(group_entry_count)

In [159]:
dgroup_val

[49,
 55,
 58,
 33,
 50,
 70,
 68,
 50,
 56,
 51,
 41,
 58,
 51,
 42,
 61,
 54,
 41,
 57,
 68,
 51,
 51,
 46,
 40,
 45,
 45,
 65,
 54,
 44,
 40,
 44,
 55,
 72,
 41,
 61,
 45,
 49,
 55,
 50,
 43,
 54,
 56,
 45,
 61,
 67,
 58,
 53,
 42,
 49,
 52,
 49,
 55,
 52,
 43,
 43,
 58,
 43,
 45,
 58,
 58,
 60,
 60,
 52,
 45,
 50,
 61,
 55,
 52,
 54,
 31,
 59,
 50,
 61,
 50,
 56,
 58,
 38,
 51,
 74,
 43,
 26,
 44,
 44,
 55,
 46,
 52,
 59,
 63,
 22,
 65,
 59,
 53,
 59,
 53,
 52,
 51,
 62,
 65,
 45,
 38,
 57,
 65,
 46,
 63,
 47,
 78,
 70,
 56,
 67,
 56,
 64,
 55,
 14,
 55,
 57,
 77,
 53,
 40,
 52,
 21,
 50,
 58,
 44,
 42,
 53,
 51,
 39,
 65,
 51,
 47,
 68,
 41,
 63,
 60,
 56,
 52,
 51,
 47,
 41,
 60,
 61,
 41,
 42,
 17,
 19,
 53,
 29,
 55,
 46,
 51,
 65,
 48,
 52,
 46,
 43,
 55,
 50,
 40,
 66,
 51,
 52,
 57,
 73,
 55,
 56,
 56,
 54,
 37,
 59,
 58,
 35,
 52,
 22,
 52,
 72,
 51,
 82,
 38,
 52,
 56,
 41,
 50,
 61,
 48,
 59,
 48,
 51,
 51,
 47,
 52,
 42,
 60,
 61,
 39,
 62,
 48,
 38,
 65,
 53,
 67,
 47,


## LambdaMart Implementation

In [160]:
lambdaMart_x_train

array([[ 0.42992772, 25.51293239,  1.11446663],
       [ 0.58244707, 25.95038068,  1.04045587],
       [ 0.27296237, 18.46453279,  1.25612974],
       ...,
       [ 0.69487517, 23.14689508,  1.07203353],
       [ 0.70842868, 23.50997488,  1.07340152],
       [ 0.64576083, 36.3284463 ,  1.11704851]])

In [161]:
dgroup_train

[63,
 53,
 64,
 62,
 48,
 56,
 57,
 54,
 50,
 57,
 57,
 50,
 40,
 48,
 48,
 54,
 62,
 50,
 65,
 44,
 47,
 54,
 39,
 44,
 49,
 54,
 45,
 59,
 39,
 66,
 55,
 53,
 47,
 58,
 53,
 51,
 61,
 49,
 49,
 51,
 15,
 44,
 51,
 50,
 59,
 39,
 61,
 54,
 46,
 61,
 48,
 49,
 52,
 22,
 39,
 48,
 46,
 52,
 56,
 55,
 58,
 43,
 44,
 74,
 59,
 48,
 40,
 51,
 60,
 51,
 50,
 49,
 58,
 45,
 43,
 60,
 43,
 45,
 46,
 53,
 58,
 43,
 22,
 26,
 31,
 65,
 46,
 44,
 47,
 63,
 57,
 53,
 37,
 47,
 66,
 49,
 58,
 32,
 65,
 55,
 59,
 72,
 48,
 48,
 56,
 57,
 35,
 56,
 52,
 70,
 50,
 68,
 39,
 56,
 50,
 49,
 47,
 44,
 43,
 56,
 55,
 43,
 57,
 55,
 57,
 20,
 47,
 56,
 58,
 53,
 57,
 53,
 46,
 55,
 68,
 43,
 24,
 61,
 68,
 50,
 44,
 43,
 61,
 49,
 56,
 57,
 50,
 68,
 50,
 65,
 57,
 68,
 68,
 43,
 34,
 60,
 30,
 63,
 54,
 53,
 54,
 58,
 68,
 54,
 48,
 47,
 51,
 45,
 64,
 47,
 55,
 41,
 54,
 58,
 64,
 61,
 60,
 47,
 38,
 49,
 49,
 54,
 54,
 52,
 48,
 41,
 57,
 63,
 55,
 64,
 65,
 45,
 52,
 50,
 45,
 66,
 66,
 44,
 62,
 63,


In [162]:
# concate Train data, very import here !

train_dmatrix = DMatrix(lambdaMart_x_train, label = lambdaMart_y_train)
valid_dmatrix = DMatrix(lambdaMart_x_val, label = lambdaMart_y_val)
train_dmatrix.set_group(dgroup_train)
valid_dmatrix.set_group(dgroup_val)




In [163]:
params = {'objective': 'rank:ndcg', 'eta': 0.1, 'gamma': 1.0,
          'min_child_weight': 0.1, 'max_depth': 6}

In [166]:
lambdaRank_model = train(params, train_dmatrix, num_boost_round=300,
                      evals=[(valid_dmatrix, 'validation')])


[0]	validation-map:0.57083
[1]	validation-map:0.59519
[2]	validation-map:0.59433
[3]	validation-map:0.59957
[4]	validation-map:0.59359
[5]	validation-map:0.59423
[6]	validation-map:0.59617
[7]	validation-map:0.59329
[8]	validation-map:0.59413
[9]	validation-map:0.59181
[10]	validation-map:0.59249
[11]	validation-map:0.58919
[12]	validation-map:0.58820
[13]	validation-map:0.59037
[14]	validation-map:0.59177
[15]	validation-map:0.59207
[16]	validation-map:0.59174
[17]	validation-map:0.59087
[18]	validation-map:0.59141
[19]	validation-map:0.59312
[20]	validation-map:0.59835
[21]	validation-map:0.60007
[22]	validation-map:0.60160
[23]	validation-map:0.59970
[24]	validation-map:0.60040
[25]	validation-map:0.60161
[26]	validation-map:0.60241
[27]	validation-map:0.60499
[28]	validation-map:0.60647
[29]	validation-map:0.60696
[30]	validation-map:0.60845
[31]	validation-map:0.60612
[32]	validation-map:0.60896
[33]	validation-map:0.60924
[34]	validation-map:0.60879
[35]	validation-map:0.61067
[3

[286]	validation-map:0.64985
[287]	validation-map:0.64983
[288]	validation-map:0.64946
[289]	validation-map:0.64956
[290]	validation-map:0.64921
[291]	validation-map:0.64933
[292]	validation-map:0.64851
[293]	validation-map:0.64854
[294]	validation-map:0.64888
[295]	validation-map:0.64884
[296]	validation-map:0.64882
[297]	validation-map:0.64835
[298]	validation-map:0.64833
[299]	validation-map:0.64819


In [167]:
test_dmatrix = valid_dmatrix

In [168]:
pred = lambdaRank_model.predict(test_dmatrix)
pred.shape
validation_data.shape

(54134,)

(54134, 10)

In [169]:
validation_data['score'] = pred
validation_data

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity,bm25,word_mover_distance,score
12997,1102335,8456194,why do people buy cars,THE COLOUR RED. RED is the colour of LOVE and ...,0.0,"[colour, red, red, colour, love, use, profusel...","[people, buy, cars]",0.660503,33.701546,1.086605,-3.125932
39497,1102335,2344898,why do people buy cars,I been wanting to buy a camshaft and on almost...,0.0,"[want, buy, camshaft, almost, ones, see, say, ...","[people, buy, cars]",0.716632,23.299768,1.064731,-2.382493
5592,1102335,4764512,why do people buy cars,"But for many people, cars are much more than t...",0.0,"[many, people, cars, much, work, art, technolo...","[people, buy, cars]",0.811969,31.370655,0.917742,-5.377942
15427,1102335,5114117,why do people buy cars,That’s why if you plan to freeze ham for more ...,0.0,"[plan, freeze, ham, months, suggest, buy, unco...","[people, buy, cars]",0.658038,31.370655,1.005135,-6.495836
53349,1102335,5953753,why do people buy cars,Bargaining or haggling is a tradition in Turke...,0.0,"[bargain, haggle, tradition, turkey, many, cou...","[people, buy, cars]",0.767158,28.044409,0.983758,-4.676255
...,...,...,...,...,...,...,...,...,...,...,...
847,2962,2694605,Explain what a bone scan is and what it is use...,Removable drives can always be scanned during ...,0.0,"[removable, drive, always, scan, quick, scan, ...","[explain, bone, scan, use]",0.723047,21.468425,0.944758,-2.219125
19557,2962,2026799,Explain what a bone scan is and what it is use...,A bone scan is a test that can find damage to ...,0.0,"[bone, scan, test, find, damage, bone, find, c...","[explain, bone, scan, use]",0.839605,35.437036,0.922435,0.024755
13767,2962,6541107,Explain what a bone scan is and what it is use...,If your doctor wants you to have a scan earlie...,0.0,"[doctor, want, scan, earlier, weeks, pregnancy...","[explain, bone, scan, use]",0.739907,19.518334,0.978086,-2.409203
16949,2962,478209,Explain what a bone scan is and what it is use...,scan with a second program after scanning with...,0.0,"[scan, second, program, scan, first, antivirus...","[explain, bone, scan, use]",0.662066,30.566273,1.001465,-6.278626


In [170]:
unique_qids_val = np.unique(validation_data.qid.values)
mean_average_precision = 0
mean_NDCG = 0
for qid in unique_qids_val:
    df_qid = validation_data[validation_data['qid'] == qid]
    df_qid_sorted = df_qid.sort_values(by=['score'], ascending=False)
    average_precision = average_precision_cal(df_qid_sorted)
    NDCG = get_NDCG(df_qid_sorted)
#     print(average_precision, NDCG)
    mean_average_precision += average_precision
    mean_NDCG += NDCG
mean_average_precision = mean_average_precision / x_val.shape[0]
mean_NDCG = mean_NDCG / x_val.shape[0]

In [171]:
mean_average_precision
mean_NDCG

0.0001043617407668995

0.005873624133531059

## Subtask 4

In [128]:
train_data = pd.read_pickle('train_data.pkl')
validation_data = pd.read_pickle('validation_data.pkl')
num_of_features = 3

In [129]:
import tensorflow as tf
from tensorflow.keras import layers, activations, losses, Model, Input
from tensorflow.nn import leaky_relu
import numpy as np
from itertools import combinations
from tensorflow.keras.utils import plot_model, Progbar
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# model architecture
class RankNet(Model):
    def __init__(self):
        super().__init__()
        self.dense = [layers.Dense(16, activation=leaky_relu), layers.Dense(8, activation=leaky_relu)]
        self.o = layers.Dense(1, activation='linear')
        self.oi_minus_oj = layers.Subtract()
    
    def call(self, inputs):
        xi, xj = inputs
        densei = self.dense[0](xi)
        densej = self.dense[0](xj)
        for dense in self.dense[1:]:
            densei = dense(densei)
            densej = dense(densej)
        oi = self.o(densei)
        oj= self.o(densej)
        oij = self.oi_minus_oj([oi, oj])
        output = layers.Activation('sigmoid')(oij)
        return output
    
    def build_graph(self):
        x = [Input(shape=(10)), Input(shape=(10))]
        return Model(inputs=x, outputs=self.call(x))

# visualize model architecture
# plot_model(RankNet().build_graph(), show_shapes=False)

In [130]:
train_data

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity,bm25,word_mover_distance
0,406386,7869580,is cider vinegar safe,2. It can make your hair shine. Apple cider vi...,0.0,"[make, hair, shine, apple, cider, vinegar, use...","[cider, vinegar, safe]",0.755494,48.331642,0.990366
1,1087603,1141888,what are field peas,"Dogs can eat raw or cooked peas, but cooked pe...",0.0,"[dog, eat, raw, cook, peas, cook, peas, softer...","[field, peas]",0.546223,25.281915,1.097962
2,115704,931826,david tepper net worth,Videos Earthquake The Comedian Net Worth 2017 ...,0.0,"[videos, earthquake, comedian, net, worth, dig...","[david, tepper, net, worth]",0.575517,50.517845,0.850333
3,1096551,8213401,how many valence electrons does an atom of the...,Those elements certainly have more than 0 vale...,0.0,"[elements, certainly, valence, electrons, fact...","[many, valence, electrons, atom, element, neon...",0.799804,68.653801,0.828285
4,490802,4649401,russ simmons net worth,Bebe Winans Net Worth. Bebe Winans Net Worth i...,0.0,"[bebe, winans, net, worth, bebe, winans, net, ...","[russ, simmons, net, worth]",0.619965,49.411318,0.896055
...,...,...,...,...,...,...,...,...,...,...
213524,660803,2204762,what foods and spices trigger vertigo,Foods that should not be included in your diet...,0.0,"[foods, include, diet, difficult, swallow, unp...","[foods, spice, trigger, vertigo]",0.583361,41.194769,1.105572
213525,228738,4411837,how does your cervix change during pregnancy,Knowledge about the position and of cervix in ...,0.0,"[knowledge, position, cervix, early, pregnancy...","[cervix, change, pregnancy]",0.732572,46.713351,0.948814
213526,1092724,7755456,midland states bank aba number,bank routing number lookup Not looking for the...,0.0,"[bank, rout, number, lookup, look, pnc, bank, ...","[midland, state, bank, aba, number]",0.747702,69.687218,0.779405
213527,1093405,4682759,beechen definition,Related searches for Operational Definitions. ...,0.0,"[relate, search, operational, definitions, exa...","[beechen, definition]",0.399225,26.360729,1.033844


In [131]:
validation_data

,qid,pid,queries,passage,relevancy,passage_cleaned,query_cleaned,co_similarity,bm25,word_mover_distance
0,405985,332147,is chatham ct incorporated,CT Angiography of the Coronary Arteries. CT An...,0.0,"[ct, angiography, coronary, arteries, ct, angi...","[chatham, ct, incorporate]",0.369007,17.481736,1.169729
1,42555,5939644,average salary for primary care sports medicin...,"In its 2011 salary survey, medical staffing fi...",0.0,"[salary, survey, medical, staff, firm, merritt...","[average, salary, primary, care, sport, medici...",0.828698,52.017312,0.841582
2,1096739,3297735,how many pounds of fat can you lose in a week,"In the calculator below, indicate your average...",0.0,"[calculator, indicate, average, number, drink,...","[many, pound, fat, lose, week]",0.770259,43.844760,0.891009
3,622725,4059964,what do decomposers and scavengers,Energy that is not used in an ecosystem is eve...,0.0,"[energy, use, ecosystem, eventually, lose, hea...","[decomposers, scavengers]",0.082339,11.233849,1.337895
4,60677,5652468,calories in one slice ham,Calories per Ingredient. Here are the foods fr...,0.0,"[calories, per, ingredient, foods, food, nutri...","[calories, one, slice, ham]",0.787347,30.288981,0.982390
...,...,...,...,...,...,...,...,...,...,...
54129,1037341,2130295,who is quinn of ny,"Centereach, NY Other Information. 1 Located i...",0.0,"[centereach, ny, information, locate, suffolk,...","[quinn, ny]",0.665690,21.000746,1.016593
54130,352236,6178442,how to cook beets without bleeding,1 Bake the beets on a baking sheet until tende...,0.0,"[bake, beets, bake, sheet, tender, pierce, kni...","[cook, beets, without, bleed]",0.752483,19.215004,1.019188
54131,991590,7301335,where is ridgewood ny located,Msg #346972. If the signing took place in NY a...,0.0,"[msg, sign, take, place, ny, ny, notary, alter...","[ridgewood, ny, locate]",0.278403,18.600281,1.188505
54132,1087675,1080515,what are deposit solutions banking,Users of Online Banking have always authentica...,0.0,"[users, online, bank, always, authenticate, ga...","[deposit, solutions, bank]",0.610685,18.947558,1.055076


In [132]:
qids = train_data.qid.values
doc_features = train_data[['co_similarity', 'bm25', 'word_mover_distance']].values
doc_scores = train_data.relevancy.values

In [133]:
qids.shape

(213529,)

In [134]:
np.unique(qids).shape

(4194,)

In [135]:
# # generate data
# nb_query = 20
# query = np.array([i+1 for i in range(nb_query) for x in range(int(np.ceil(np.abs(np.random.normal(0,scale=15))+2)))])
# # doc_features = np.random.random((len(query), 10))
# # doc_scores = np.random.randint(5, size=len(query)).astype(np.float32)
# query

In [136]:
# np.where(query == 1)

In [137]:
# put data into pairs
xi = []
xj = []
pij = []
pair_id = []
pair_query_id = []
for qid in np.unique(qids):
    query_idx = np.where(qids == qid)[0]
#     print(query_idx)
    for pair_idx in combinations(query_idx, 2):
        pair_query_id.append(qid)
        
        pair_id.append(pair_idx)
        i = pair_idx[0]
        j = pair_idx[1]
        xi.append(doc_features[i])
        xj.append(doc_features[j])
        
        if doc_scores[i] == doc_scores[j]:
            _pij = 0.5
        elif doc_scores[i] > doc_scores[j]:
            _pij = 1
        else: 
            _pij = 0
        pij.append(_pij)
        
xi = np.array(xi)
xj = np.array(xj)
pij = np.array(pij)
pair_query_id = np.array(pair_query_id)


xi_train, xi_test, xj_train, xj_test, pij_train, pij_test, pair_id_train, pair_id_test = train_test_split(
    xi, xj, pij, pair_id, test_size=0.2)


In [138]:
# train model using compile and fit
ranknet = RankNet()
ranknet.compile(optimizer='adam', loss='binary_crossentropy')
history = ranknet.fit([xi_train, xj_train], pij_train, epochs=50, batch_size=1, validation_data=([xi_test, xj_test], pij_test))


Train on 4480318 samples, validate on 1120080 samples
Epoch 1/50
  40676/4480318 [..............................] - ETA: 1:40:17 - loss: 0.6929

KeyboardInterrupt: 